In [1]:
import torch
from torch.utils.data import Dataset
from PIL import Image
import os
import torch.nn as nn
from torch.nn import functional as F
from torchvision import transforms
from torch.utils.data import DataLoader
import torch.optim as optim
from torchvision.utils import save_image
import cv2
import numpy as np
from torch.nn.functional import one_hot
import tensorflow as tf
from torch.utils.tensorboard import SummaryWriter   
transform = transforms.Compose([
    transforms.ToTensor()
])
class MyDataset(Dataset):#制作数据集
    def __init__(self,path):
        self.path=path
        self.name=os.listdir('D:/mask/')#读取名称
        
    def __len__(self):
        return len(self.name)
    
    def __getitem__(self,index):
        m_name=self.name[index]#读取名称
        m_path=os.path.join(self.path,'mask',m_name)#mask的地址
        i_path=os.path.join(self.path,'lianhua',m_name)#原图的地址
        i_image=Image.open(i_path)
        i_image = np.array(i_image)
        #i_image=self.keep_image_size_open_rgb(i_path)
        m_image=Image.open(m_path)
        m_image = np.array(m_image)
        #cv2.imwrite('kkk.png', m_image)
        return transform(i_image),transform(m_image)
"""
data=MyDataset('D:/')
data1=DataLoader(dataset=data,batch_size=1)
for i ,data in enumerate(data1):
    x,y=data
    #print(x.shape)
    print(len(y[y==255]))
    print(y.shape)
"""

"\ndata=MyDataset('D:/')\ndata1=DataLoader(dataset=data,batch_size=1)\nfor i ,data in enumerate(data1):\n    x,y=data\n    #print(x.shape)\n    print(len(y[y==255]))\n    print(y.shape)\n"

In [68]:
#调整图片尺寸
from PIL import Image
import os.path
import glob
def convertjpg(jpgfile,outdir,width=128,height=128):
    img=Image.open(jpgfile)
    try:
        new_img=img.resize((width,height),Image.BILINEAR)   
        new_img.save(os.path.join(outdir,os.path.basename(jpgfile)))
    except Exception as e:
        print(e)
for jpgfile in glob.glob("D:\\mas\\*.png"):
    convertjpg(jpgfile,"D:\\mas")

In [21]:
#调整mask像素值
from PIL import Image
import os.path
import glob
def convertjpg1(jpgfile,outdir):
    img=Image.open(jpgfile)
    new_img=np.array(img)*255.0  
    cv2.imwrite(os.path.basename(jpgfile), new_img)
        #new_img.save(outdir,os.path.basename(jpgfile))
for jpgfile in glob.glob("D:\\jupyter\\mas\\*.png"):
    convertjpg1(jpgfile,"")   

In [2]:
class DConv(nn.Module):#卷积块
    def __init__(self,in_channels,out_channels):
        super(DConv,self).__init__()
        self.conv = nn.Sequential(
          nn.Conv2d(in_channels,out_channels,3,1,1,bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(inplace=True),
          nn.Conv2d(out_channels, out_channels ,3,1,1,bias=False),
          nn.BatchNorm2d(out_channels),
          nn.ReLU(inplace=True),
        )
    def forward(self,x):
        return self.conv(x)
class up(nn.Module):
    def __init__(self,in_channels,out_channels):
        super(up,self).__init__()
        self.up=nn.ConvTranspose2d(in_channels,in_channels//2,2,2)
        self.conv = DConv(in_channels,out_channels)
    def forward(self,x1,x2):
        x1 = self.up(x1)
        diffY = x2.size()[2] - x1.size()[2]
        diffX = x2.size()[3] - x1.size()[3]

        x1 = F.pad(x1, [diffX // 2, diffX - diffX // 2,
                        diffY // 2, diffY - diffY // 2])
        x = torch.cat([x2, x1], dim=1)
        return self.conv(x)
    
class U_Net(nn.Module):
    def __init__(self,n_classes):#(2,3,256,256)
        super(U_Net, self).__init__()
        self.con1=DConv(3,64)#(2,64,256,256)
        self.m1=nn.MaxPool2d(2)#(2,64,128,128)
        self.con2=DConv(64,128)#(2,128,128,128)
        self.m2=nn.MaxPool2d(2)#(2,128,64,64)
        self.con3=DConv(128,256)#(2,256,64,64)
        self.m3=nn.MaxPool2d(2)#(2,256,32,32)
        self.con4=DConv(256,512)#(2,512,32,32)

        self.m4=nn.MaxPool2d(2)#(2,512,16,16)
        self.con5=DConv(512,1024)#(2,1024,16,16)
        #Upscaling
        self.up1=up(1024,512)#(2,512,32,32)
        self.up2=up(512,256)
        self.up3=up(256,128)
        self.up4=up(128,64)
        self.out=nn.Conv2d(64,n_classes,1)
        
    def forward(self,x):
        x1=self.con1(x)
        x2=self.con2(self.m1(x1))#(2,128,128,128)
        x3=self.con3(self.m2(x2))#(2,256,64,64)
        x4=self.con4(self.m3(x3))#(2,512,32,32)
        x5=self.con5(self.m4(x4))#(2,1024,16,16)
        x=self.up1(x5,x4)#(2,512,32,32)
        x=self.up2(x,x3)#(2,256,64,64)
        x=self.up3(x,x2)#(2,128,128,128)
        x=self.up4(x,x1)#(2,64,256,256)
        
        return torch.sigmoid(self.out(x))#(2,1,256,256)

In [3]:
"""
class DiceLoss(nn.Module):
    def __init__(self):
        super(DiceLoss, self).__init__()
    def forward(self, input, target):
        N = target.size(0)
        smooth = 1
        input_flat = input.view(N, -1)
        target_flat = target.view(N, -1)
        intersection = input_flat * target_flat
        loss = 2 * (intersection.sum(1) + smooth) / (input_flat.sum(1) + target_flat.sum(1) + smooth)
        loss = 1 - loss.sum() / N
        return loss

"""
"""
class SoftDiceLoss(nn.Module):
    def __init__(self, weight=None, size_average=True):
        super(SoftDiceLoss, self).__init__()
        
    def forward(self, logits, targets):
        num = targets.size(0)
        smooth = 0.75
        
        probs = F.sigmoid(logits)
        m1 = probs.view(num, -1)
        m2 = targets.view(num, -1)
        intersection = (m1 * m2)
        score = 2. * (intersection.sum(1) + smooth) / (m1.sum(1) + m2.sum(1) + smooth)
        score = 1 - score.sum() / num
        return score

"""
"""
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.logits = logits
        self.reduce = reduce

    def forward(self, inputs, targets):
        if self.logits:
            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)
        else:
            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)
        pt = torch.exp(-BCE_loss)
        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss

        if self.reduce:
            return torch.mean(F_loss)
        else:
            return F_loss
"""

'\nclass FocalLoss(nn.Module):\n    def __init__(self, alpha=1, gamma=2, logits=False, reduce=True):\n        super(FocalLoss, self).__init__()\n        self.alpha = alpha\n        self.gamma = gamma\n        self.logits = logits\n        self.reduce = reduce\n\n    def forward(self, inputs, targets):\n        if self.logits:\n            BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduce=False)\n        else:\n            BCE_loss = F.binary_cross_entropy(inputs, targets, reduce=False)\n        pt = torch.exp(-BCE_loss)\n        F_loss = self.alpha * (1-pt)**self.gamma * BCE_loss\n\n        if self.reduce:\n            return torch.mean(F_loss)\n        else:\n            return F_loss\n'

In [6]:
BATCH_SIZE=32
LR=0.0001
Epoch=15
mini_batch=5
loss_fn=nn.BCELoss()
#loss_fn=DiceLoss()
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
net=U_Net(1).to(device)
data_path=r'D:\\'
optimizer=optim.Adam(net.parameters(), lr=LR, betas=(0.9, 0.999), eps=1e-8, weight_decay=0)
data_loader=DataLoader(MyDataset(data_path),batch_size=BATCH_SIZE,shuffle=True)
data_ = iter(data_loader)
(inputs, labels) = next(data_)
print(inputs.shape)
print(labels.shape)


print('Start training')
s_path='D:\jupyter\\train_image'

torch.Size([32, 3, 128, 128])
torch.Size([32, 1, 128, 128])
Start training


In [7]:
import numpy as np
def hist_info(n_cl, pred, gt):
    assert (pred.shape == gt.shape)  # gt的形状和pred的形状必须相等
    k = (gt >=0)  # 生成一个H×W的mask，里面类别属于0~n_cl的为True，否则为False
    labeled = np.sum(k)  # 统计mask中元素为True的数量，即标签类别是0~n_cl的元素个数
    correct = np.sum((pred[k] == gt[k]))  # 统计预测正确的数量
    print(labeled)
    print(correct)
    # 返回混淆矩阵、统计元素的数量和预测正确元素的数量。
    return np.bincount(n_cl * gt[k].astype(int) + pred[k].astype(int),
                       minlength=n_cl ** 2).reshape(n_cl, n_cl), labeled, correct

def compute_score(hist, correct, labeled): 
    iu = np.diag(hist) / (hist.sum(1) + hist.sum(0) - np.diag(hist))  # 交并比 = 对角线元素 / 所有元素和 -对角线元素

    mean_IU = np.nanmean(iu)  # numpy.nanmean()函数可用于计算忽略NaN值的数组平均值,经过这一步就算得了MIOU
    mean_IU_no_back = np.nanmean(iu[1:])  # 除去背景的MIOU（一般情况下0类别代表背景）
    mean_pixel_acc = correct / labeled  # 平均像素准确率 = 预测正确像素个数 / 总像素个数
    return iu, mean_IU, mean_IU_no_back, mean_pixel_acc
#output_m=np.array([[0,1,2],[0,0,1],[2,0,1]])
#a,b,bc=hist_info(1, output_m, mask.detach().numpy())

In [8]:
import numpy as np
loss_dict = []
iou_dict=[]
threshold, upper, lower=0.5,1,0
best_m=0.0
tb = SummaryWriter()
for epoch in range(Epoch): 
    for i,(image,mask) in enumerate(data_loader):
        image=image.to(device)
        mask=mask.to(device)
        print(len(mask.numpy()[mask==1]))
        output=net(image)
        output_m=output
        #print(len(output.detach().numpy().astype(np.int8)[output==255]))
        output_m=np.where(output>threshold, upper, lower)
        #print(len(output_m.astype(np.int8)[output_m==255]))
       # print(len(mask.numpy()[mask==255]))
        #print(len(output_m.astype(np.int8)[output_m==0]))
        hist,labeled, correct=hist_info(2, output_m, mask.detach().numpy())
        iu, mean_IU, mean_IU_no_back, mean_pixel_acc= compute_score(hist, correct, labeled)
        loss = loss_fn(output,mask)
        tb.add_scalar('loss/Diceloss', loss, epoch)
        tb.close()
        print('mIoU is : %f' % mean_IU)
        optimizer.zero_grad()
        loss.backward()                     # 梯度反向传播
        optimizer.step()                    # 优化器优化参
        
        loss_dict.append(loss.item())
        if (epoch+1) % mini_batch == 0:
            print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, Epoch, loss.item()))
        if mean_IU > best_m:
            torch.save(net, "unet_methoddice.pth")
            print("save!")

            
        #看效果
        _image=image[0]
        _mask=mask[0]
        _out_image=output[0]
        save_image(_out_image,f'{s_path}/i.png')

    epoch+=1
print("EDN")
                  

137499
524288
228631
mIoU is : 0.270646
save!
127436
524288
278534
mIoU is : 0.352597
save!
128803
524288
307611
mIoU is : 0.400069
save!
118268
524288
343010
mIoU is : 0.460731
save!
137798
524288
339852
mIoU is : 0.458170
save!
128078
524288
348885
mIoU is : 0.470251
save!
131937
524288
367507
mIoU is : 0.512687
save!
118143
524288
402621
mIoU is : 0.584290
save!
96302
360448
273613
mIoU is : 0.585277
save!
130225
524288
422810
mIoU is : 0.643077
save!
129759
524288
396778
mIoU is : 0.575525
save!
124951
524288
439830
mIoU is : 0.682563
save!
129564
524288
433719
mIoU is : 0.667168
save!
131121
524288
457033
mIoU is : 0.732800
save!
125064
524288
450242
mIoU is : 0.708399
save!
144496
524288
444375
mIoU is : 0.699019
save!
126811
524288
463304
mIoU is : 0.749423
save!
82273
360448
327001
mIoU is : 0.784614
save!
127851
524288
475970
mIoU is : 0.789878
save!
122131
524288
477945
mIoU is : 0.793187
save!
122550
524288
479252
mIoU is : 0.796952
save!
134865
524288
471271
mIoU is : 0.775

In [9]:
class MyDataset1(Dataset):#制作数据集
    def __init__(self,path):
        self.path=path
        self.name=os.listdir('D:/jupyter/lianhua/')#读取名称
        
    def __len__(self):
        return len(self.name)
    
    def __getitem__(self,index):
        m_name=self.name[index]#读取名称
        m_path=os.path.join(self.path,'mask',m_name)#mask的地址
        i_path=os.path.join(self.path,'lianhua',m_name)#原图的地址
        i_image=Image.open(i_path)
        i_image = np.array(i_image)
        #i_image=self.keep_image_size_open_rgb(i_path)
        m_image=Image.open(m_path)
        m_image = np.array(m_image)
        #cv2.imwrite('kkk.png', m_image)
        return transform(i_image),transform(m_image)

In [10]:
BATCH_SIZE=49
LR=0.0001
Epoch=1
mini_batch=5

#loss_fn=nn.BCEWithLogitsLoss()
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
data_path=r'D:\\jupyter'
data_loader1=DataLoader(MyDataset1(data_path),batch_size=BATCH_SIZE,shuffle=True)
data_1 = iter(data_loader1)
(inputs, labels) = next(data_1)
print(inputs.shape)
print(labels.shape)


print('Start testing')
s_path='D:\jupyter\\test_image'

torch.Size([49, 3, 128, 128])
torch.Size([49, 1, 128, 128])
Start testing


In [16]:
print('Started testing')
#model = torch.load("unet_method1.pth")
model = torch.load("unet_methoddice.pth")
with torch.no_grad():   #测试阶段，不需要梯度反传，因此不需要保存梯度信息
    for i,(image,mask) in enumerate(data_loader1):
        image=image.to(device)
        mask=mask.to(device)
        output=model(image)
        output_m=output
        #print(len(output.detach().numpy().astype(np.int8)[output==255]))
        output_m=np.where(output>threshold, upper, lower)
        #print(len(output_m.astype(np.int8)[output_m==255]))
       # print(len(mask.numpy()[mask==255]))
        #print(len(output_m.astype(np.int8)[output_m==0]))
        hist,labeled, correct=hist_info(2, output_m, mask.detach().numpy())
        iu, mean_IU, mean_IU_no_back, mean_pixel_acc= compute_score(hist, correct, labeled)
        print('mIoU is : %f' % mean_IU)
        print('mean_pixel_acc is : %f' % mean_pixel_acc)
        #看效果
        _image=image[0]
        _mask=mask[0]
        _out_image=output[0]
        save_image(_out_image,f'{s_path}/i.png')
        #cv2.imwrite('kmm.png',_out_image)
        save_image(_image,f'{s_path}/m.png')
        save_image(_mask,f'{s_path}/n.png')

        epoch+=1
print("EDN")

Started testing
802816
760783
mIoU is : 0.877330
mean_pixel_acc is : 0.947643
EDN
